2) Un importante servicio de monitoreo de aplicaciones cloud, sufrió un importante incidente en las últimas semanas por lo cual está conduciendo una investigación. Nuestro equipo participa en el análisis de forma externa para lo cual nos provee los siguientes dataframes en csv:
- metrics.csv describiendo información de métricas obtenidas. El mismo tiene el formato ('client_id', 'integration_id' ,'metric', 'timestamp', 'value') donde la columna ‘metric’ indica el nombre de una métrica que se registra en un cierto momento ('timestamp') cuyo valor se guarda en la columna 'value'.
- clients.csv con información sobre el cliente, con el formato ('client_id, 'account_number', 'name') donde 'account_number' indica el número de cuenta de cliente y 'name' el nombre del mismo.
Como miembro de nuestro equipo es necesario que obtengas las siguientes métricas para aportar insights a nuestro análisis:


a) Calcular el valor promedio de las metricas 'aws.vpc.network_out', 'aws.vpc.network_in', 'aws.vpc.network_rate' para los clientes de ‘account_number’ mayor al '25679247' devolviendo el resultado en el siguiente formato (account_number, 'aws.vpc.network_out_mean', 'aws.vpc.network_in_mean', 'aws.vpc.network_rate_mean')(15pts)


b) Calcular la diferencia entre el valor promedio obtenido en cada métrica por cada cliente, y el valor promedio general de esa métrica (sin utilizar group by) (10pts)


metrics.csv describe informacion de las metricas obtenidas 
formato: ('client_id','integration_id','metric','timestamp','value')
'metric' indica el nombre de una metrica que se registra en un determinado momento 'timestamp' cuyo valor se guarda en la columna 'value'.

clients.csv tiene informacion del cliente
formato: ('client_id','account_number','name')
'account_number' indica el numero de cuenta de cliente y 'name' el nombre del mismo

In [3]:
import pandas as pd 
import numpy as np

In [7]:
#creo datos random para probar
dataMetrics = {'client_id': [1,2,3,4,5,6,7,8,9,10,3,2,6,15], \
         'integration_id': [1,1,1,1,1,1,1,1,1,1,1,1,1,1],\
         'metric': ['aws.vpc.network_out','aws.vpc.network_out','aws.vpc.network_in','aws.vpc.network_rate','aws.vpc.network_out','aws.vpc.network_in','aws.vpc.network_in','aws.vpc.network_rate','aws.vpc.network_rate','aws.vpc.network_in','aws.vpc.network_in','aws.vpc.network_random','aws.vpc.network_rate','aws.vpc.network_out'],\
         'timestamp': ['2021-01-01','2021-01-01','2021-01-01','2021-01-01','2021-01-01','2021-01-01','2021-01-01','2021-01-01','2021-01-01','2021-01-01','2021-01-01','2021-01-01','2021-01-01','2021-01-01'],\
         'value': [6,7,8,9,10,1,2,3,4,7,6,5,4,3]
        }

In [8]:
#creo datos random para probar
dataClients = {'client_id': [3,2,6,15,10,9,8,7,6,1,2,3,4,5],\
              'account_number': [103245,1679800,25679247,30867590,28312420,31890652,45987751,25700000,27650931,25300654,28888555,24567543,28980761,51001200],\
               'name': ['pedro','juan','gonzalo','maria','josefina','esteban','celeste','ramiro','juana','giovani','pilar','yamil','luis','ayelen']
              }

In [ ]:
#primero leo los .csv
#metrics_df = pd.read_csv(metrics.csv)
#clients_df = pd.read_csv(clients.csv)

In [6]:
metrics_df = pd.DataFrame(data = dataMetrics)
metrics_df.sample(3)

,client_id,integration_id,metric,timestamp,value
9,10,1,aws.vpc.network_in,2021-01-01,7
0,1,1,aws.vpc.network_out,2021-01-01,6
4,5,1,aws.vpc.network_out,2021-01-01,10


In [15]:
clients_df = pd.DataFrame(data = dataClients)
clients_df.sample(3)

,client_id,account_number,name
9,1,25300654,giovani
0,3,103245,pedro
11,3,24567543,yamil


In [18]:
#primero fitlro ols clientes 
filtered_clients = clients_df.loc[clients_df['account_number'] > 25679247]

In [26]:
#filtro tambien las metricas deseadas
metrics_of_interest = (metrics_df['metric'] == 'aws.vpc.network_out') | (metrics_df['metric'] == 'aws.vpc.network_in') \
                        | (metrics_df['metric'] == 'aws.vpc.network_rate')
filtered_metrics = metrics_df.loc[metrics_of_interest]
#ya filtrado las columnas que me interesan son 'client_id', 'metric' y 'value'
filtered_metrics = filtered_metrics[['client_id', 'metric', 'value']]
filtered_metrics.sample(4)

,client_id,metric,value
5,6,aws.vpc.network_in,1
12,6,aws.vpc.network_rate,4
6,7,aws.vpc.network_in,2
4,5,aws.vpc.network_out,10


In [92]:
#mergeo con el metrics_df 
joined_df = filtered_metrics.merge(filtered_clients, on= 'client_id', how = 'inner')
joined_df

,client_id,metric,value,account_number,name
0,2,aws.vpc.network_out,7,28888555,pilar
1,4,aws.vpc.network_rate,9,28980761,luis
2,5,aws.vpc.network_out,10,51001200,ayelen
3,6,aws.vpc.network_in,1,27650931,juana
4,6,aws.vpc.network_rate,4,27650931,juana
5,7,aws.vpc.network_in,2,25700000,ramiro
6,8,aws.vpc.network_rate,3,45987751,celeste
7,9,aws.vpc.network_rate,4,31890652,esteban
8,10,aws.vpc.network_in,7,28312420,josefina
9,15,aws.vpc.network_out,3,30867590,maria


In [116]:
result_df = joined_df.pivot_table(index= 'account_number', columns='metric', values= 'value', aggfunc='mean').reset_index()
result_df = result_df.rename(columns = {'aws.vpc.network_in':'aws.vpc.network_in_mean','aws.vpc.network_out':'aws.vpc.network_out_mean','aws.vpc.network_rate':'aws.vpc.network_rate_mean'})
result_df.rename_axis(None, axis = 1, inplace = True)
result_df
#para determinadas account_number hay metricas que no se midieron, por ende voy a tener valores NaN

,account_number,aws.vpc.network_in_mean,aws.vpc.network_out_mean,aws.vpc.network_rate_mean
0,25700000,2.0,NaN,NaN
1,27650931,1.0,NaN,4.0
2,28312420,7.0,NaN,NaN
3,28888555,NaN,7.0,NaN
4,28980761,NaN,NaN,9.0
5,30867590,NaN,3.0,NaN
6,31890652,NaN,NaN,4.0
7,45987751,NaN,NaN,3.0
8,51001200,NaN,10.0,NaN


Para la parte b:

In [122]:
average_in = result_df['aws.vpc.network_in_mean'].mean()
average_out = result_df['aws.vpc.network_out_mean'].mean()
average_rate = result_df['aws.vpc.network_rate_mean'].mean()

result_df['diff_in'] = result_df['aws.vpc.network_in_mean'].apply(lambda x: x - average_in)
result_df['diff_out'] = result_df['aws.vpc.network_out_mean'].apply(lambda x: x - average_out)
result_df['diff_rate'] = result_df['aws.vpc.network_rate_mean'].apply(lambda x: x - average_rate)

result_df

,account_number,aws.vpc.network_in_mean,aws.vpc.network_out_mean,aws.vpc.network_rate_mean,diff_out,diff_in,diff_rate
0,25700000,2.0,NaN,NaN,NaN,-1.333333,NaN
1,27650931,1.0,NaN,4.0,NaN,-2.333333,-1.0
2,28312420,7.0,NaN,NaN,NaN,3.666667,NaN
3,28888555,NaN,7.0,NaN,0.333333,NaN,NaN
4,28980761,NaN,NaN,9.0,NaN,NaN,4.0
5,30867590,NaN,3.0,NaN,-3.666667,NaN,NaN
6,31890652,NaN,NaN,4.0,NaN,NaN,-1.0
7,45987751,NaN,NaN,3.0,NaN,NaN,-2.0
8,51001200,NaN,10.0,NaN,3.333333,NaN,NaN
